In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')


Processing-part-1: extracting raw table

In [3]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue

    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

part-2: integrating Postal codes with more than 1 neighbour

In [6]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 78


creating an appropriate DF

In [8]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M3K,North York,CFB Toronto
1,M2J,North York,Henry Farm
2,M5R,Central Toronto,"The Annex, Yorkville"
3,M6H,West Toronto,Dovercourt Village
4,M1G,Scarborough,Woburn
5,M7A,Downtown Toronto,Queen's Park
6,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
7,M1S,Scarborough,Agincourt
8,M4H,East York,Thorncliffe Park
9,M4B,East York,"Woodbine Gardens, Parkview Hill"


In [10]:
df_toronto.shape

(78, 3)

In [12]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coordenadas = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [14]:
print(coordenadas.shape)
coordenadas.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_df = df_toronto.set_index('Postcode')
coordenadas_df = coordenadas.set_index('Postal Code')
toronto_final = pd.concat([toronto_df, coordenadas_df], axis=1, join='inner')

toronto_final.index.name = 'Postcode'
toronto_final.reset_index(inplace=True)

print(toronto_final.shape)
toronto_final.head()

(78, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3K,North York,CFB Toronto,43.737473,-79.464763
1,M2J,North York,Henry Farm,43.778517,-79.346556
2,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678
3,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
4,M1G,Scarborough,Woburn,43.770992,-79.216917
